In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("SparkHive2")\
    .config("spark.sql.warehouse.dir","hdfs://node120:9000/hive/warehouse")\
    .config("hive.metastore.uris","thrift://node120:9083")\
    .enableHiveSupport()\
    .getOrCreate()

### 全局视图

In [3]:
res1 = spark.read.format("csv").load('../data/housing.csv')

In [5]:
res1.createGlobalTempView("housing")

In [ ]:
spark.sql("SELECT * FROM global_temp.housing").show()

### 自定义函数UDF

In [4]:
df1 = spark.read.format("json").load('../data/user.jsons')

In [5]:
df1.show()

+-------+---+--------------+-----+
|address|age|       hobbies| name|
+-------+---+--------------+-----+
|   null| 19|[游戏, 羽毛球]|张三1|
|   null| 18|[篮球, 羽毛球]|张三2|
|   null| 17|[小说, 羽毛球]|张三3|
|   null| 19|[电视, 羽毛球]|张三4|
|   null| 17|  [游戏, 电影]|张三5|
|   null| 18|  [游戏, 篮球]|张三6|
| [郑州]| 19|[小说, 乒乓球]|张三7|
+-------+---+--------------+-----+



In [8]:
from pyspark.sql.types import IntegerType

# 创建临时视图
df1.createTempView("users")

def multiply_10(number):
    return number * 10
# 注册函数
udf2 = spark.udf.register("udf1", multiply_10, IntegerType())
# 使用自定义函数
df1.select(udf2(df1.age)).show()

+---------+
|udf1(age)|
+---------+
|      190|
|      180|
|      170|
|      190|
|      170|
|      180|
|      190|
+---------+



In [9]:
spark.sql("select udf1(age) from users").show()

+---------+
|udf1(age)|
+---------+
|      190|
|      180|
|      170|
|      190|
|      170|
|      180|
|      190|
+---------+



In [10]:
spark.sql("select age from users").show()

+---+
|age|
+---+
| 19|
| 18|
| 17|
| 19|
| 17|
| 18|
| 19|
+---+

